In [3]:
import pandas as pd
import numpy as np
import warnings 
import os
warnings.filterwarnings('ignore')

In [4]:
filepath=os.getcwd()

In [6]:
train_review = pd.read_json(filepath+'/devided_dataset_v2/Toys_and_Games/train/review_training.json')
train_product = pd.read_json(filepath+'/devided_dataset_v2/Toys_and_Games/train/product_training.json')

In [180]:
test_review = pd.read_json(filepath+'/devided_dataset_v2/Toys_and_Games/test1/review_test.json')
test_product = pd.read_json(filepath+'/devided_dataset_v2/Toys_and_Games/test1/product_test.json')

In [14]:
train_product.head()

,asin,awesomeness
0,000254CDA01E5AE839E7EE3F6FC1D2F1,1
1,00027761289A98A2884B5F152D3DBF91,0
2,0002E81AED1B9840434EE28E39876BE4,0
3,0003806D9BE7957D8AB9AF9EC04F52D0,1
4,00049B538C925B3EF702FFF6FBD48DFE,1


In [15]:
id_counts = train_review['reviewerID'].value_counts()
user_id_dict = id_counts.to_dict()
train_review['reviewerNumber'] = train_review['reviewerID'].map(user_id_dict)

In [67]:
train = train_review.merge(train_product, left_on = 'asin', right_on = 'asin')
train.head()

,asin,reviewerID,unixReviewTime,verified,reviewTime,reviewerName,reviewText,summary,vote,image,style,reviewerNumber,awesomeness
0,52BB606CF3F81D1F307C3EE312D387A8,128D3FF453A2E03BB746294BABD513C0,1416528000,True,"11 21, 2014",06DFD3AEA5930F6A3FF093A43CE388CC,These are fun and colourful recommended for an...,Five Stars,None,None,None,3,1
1,52BB606CF3F81D1F307C3EE312D387A8,B08B9444C17ED3A10A0E4EE673FD8241,1432425600,True,"05 24, 2015",563B3366BB6B5B40A798B83255049D8D,Granddaughter loved it and was delivered timel...,"Why is sells, I can't imagine.",None,None,None,3,1
2,52BB606CF3F81D1F307C3EE312D387A8,A0A84CC1459C9B04AB83A887E065F5C2,1427414400,True,"03 27, 2015",3A3EA00CFC35332CEDF6E5E9A32E94DA,My 6 year old daughter loves it.,Five Stars,None,None,None,15,1
3,52BB606CF3F81D1F307C3EE312D387A8,80B2267130CFD9630B67F57DB75CD022,1459900800,True,"04 6, 2016",760C63E8E5E8DC3FAA01878D37BA5678,"Packed well, nice quality, my sister loves it:)","nice quality, my sister loves it",None,None,None,2,1
4,52BB606CF3F81D1F307C3EE312D387A8,3F42FFC440D870A86A222519CD4836A3,1406678400,True,"07 30, 2014",6917F0F6BD3F74D87149D44B847682EE,Just what my daugther wanted.,Perfect,None,None,None,4,1


# STOP WORD REMOVAL 

In [46]:
#only do this once 
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('wordnet')

In [83]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

lemmatizer = WordNetLemmatizer()
stop_words = {word.lower() for word in stopwords.words('english')}
punctuation_regex = re.compile(r'[^\w\s]')

def remove_stopwords(text):
    # Remove punctuation
    text = punctuation_regex.sub('', text)
    # remove all single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    # remove all the special characters
    text = re.sub(r'\W', ' ', str(text))
    # Remove single characters from the start
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)
    text= re.sub('".*?"', '', text) #removing any word within quotation marks
    # Tokenize the text
    words = word_tokenize(text)
    # Remove stop words and lemmatize
    filtered_words = (lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words)
    
    
    # Join the filtered words back into a string
    return ' '.join(filtered_words)

In [84]:
train['reviewText'] = train['reviewText'].astype(str)
train['reviewText']=train['reviewText'].apply(remove_stopwords)
train['summary'] = train['summary'].astype(str)
train['summary']=train['summary'].apply(remove_stopwords)

In [86]:
train

,asin,reviewerID,unixReviewTime,verified,reviewTime,reviewerName,reviewText,summary,vote,image,style,reviewerNumber,awesomeness
0,52BB606CF3F81D1F307C3EE312D387A8,128D3FF453A2E03BB746294BABD513C0,1416528000,True,"11 21, 2014",06DFD3AEA5930F6A3FF093A43CE388CC,fun colourful recommended little girl collection,five star,None,None,None,3,1
1,52BB606CF3F81D1F307C3EE312D387A8,B08B9444C17ED3A10A0E4EE673FD8241,1432425600,True,"05 24, 2015",563B3366BB6B5B40A798B83255049D8D,granddaughter loved delivered timely promised,sell cant imagine,None,None,None,3,1
2,52BB606CF3F81D1F307C3EE312D387A8,A0A84CC1459C9B04AB83A887E065F5C2,1427414400,True,"03 27, 2015",3A3EA00CFC35332CEDF6E5E9A32E94DA,6 year old daughter love,five star,None,None,None,15,1
3,52BB606CF3F81D1F307C3EE312D387A8,80B2267130CFD9630B67F57DB75CD022,1459900800,True,"04 6, 2016",760C63E8E5E8DC3FAA01878D37BA5678,packed well nice quality sister love,nice quality sister love,None,None,None,2,1
4,52BB606CF3F81D1F307C3EE312D387A8,3F42FFC440D870A86A222519CD4836A3,1406678400,True,"07 30, 2014",6917F0F6BD3F74D87149D44B847682EE,daugther wanted,perfect,None,None,None,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
988401,76B1E4CFEDB3C68CE50CE8F9E4A14DF6,1B65F8E438C22DE74C18697F6CA2FCA5,1430092800,True,"04 27, 2015",D80517C8069D7702D8FDD89B64B4ED3B,every family greatest diversity gift,greatest diversity gift,None,None,None,11,0
988402,D788EA4B8BFD70DDCD6A754E18646A64,4438BF13B3B76E97DC2FAECA3FC1EB80,1514419200,True,"12 28, 2017",5D8716BEECEE1105AB6847F779B6DE70,perfect price made rubberplastic stood dog get...,perfect price,None,None,None,2,1
988403,06095E33D2AF4D8B5B50F159B30085B6,4CC1D9BF24B927B80155899AADE99F98,1518480000,True,"02 13, 2018",416C841B23B1BF083BD2A304BE58A488,present grandson enjoyed playing seems pretty ...,fun without battery,None,None,None,3,1
988404,231D30C77E01A2D7112CEDF4542367BC,D7567AA598E9BA659CBF6072A9D4CBCA,1477008000,True,"10 21, 2016",DB05833C29E688B5AB54D5E8608A72EC,good toy,delivery service took long,None,None,None,6,0


In [129]:
#need to pip install vaderSentiment in terminal
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def get_sentiment_score(text):
    sentiment_scores = analyzer.polarity_scores(text)
    return sentiment_scores['compound']

# apply the function to the 'reviewText' column: takes ~5-10 min to run 
train['sentiment_score_review'] = train['reviewText'].apply(get_sentiment_score)
train['sentiment_score_summary'] = train['summary'].apply(get_sentiment_score)

KeyError: 'reviewText'

In [130]:
#create star sentiment based on summary
def star_sentiment(summary):
    if 'five star' in summary:
        return 1
    elif 'four star' in summary or 'three star' in summary or 'two star' in summary or 'one star' in summary:
        return -1
    else:
        return 0

# Apply the lambda function to the 'summary' column and create a new column 'star_sentiment'
train['star_sentiment'] = train['summary'].apply(lambda x: star_sentiment(x))
train

KeyError: 'summary'

In [254]:
#changed verified to 1 and 0, change vote none to 0, change image to binary
train["verified"] = train["verified"].astype(int)
train["vote"] = train["vote"].replace(np.nan,0) 
train['image'] = train['image'].fillna(0)
train['image'] = train['image'].apply(str)
train['image'] = train['image'].replace(to_replace=r'[^0-9]', value=1, regex=True)

In [156]:
train = train[['asin', 'verified', 'vote', 'image', 'reviewerNumber', 'awesomeness', 'sentiment_score_review', 'sentiment_score_summary', 'star_sentiment']] 
#Change to float
train['vote'] = train['vote'].apply(str)
train['vote'] = train['vote'].apply(lambda x: float(x.replace(',', '')))
train['image'] = train['image'].astype(float)

In [451]:
train_counts = train.groupby('asin').size().reset_index(name='num_reviews_per_product')
train_final = train.groupby('asin').mean().reset_index().merge(train_counts, on='asin')
train_final.head()

,asin,verified,vote,reviewerNumber,awesomeness,sentiment_score_review,sentiment_score_summary,star_sentiment,num_reviews_per_product
0,000254CDA01E5AE839E7EE3F6FC1D2F1,1.000000,0.300000,4.900000,1.0,0.437310,0.307800,0.200000,10
1,00027761289A98A2884B5F152D3DBF91,1.000000,1.000000,7.750000,0.0,0.341650,-0.090300,0.000000,4
2,0002E81AED1B9840434EE28E39876BE4,0.833333,0.000000,4.833333,0.0,-0.116767,0.136517,-0.166667,6
3,0003806D9BE7957D8AB9AF9EC04F52D0,0.400000,0.000000,5.400000,1.0,0.635560,0.441300,0.000000,5
4,00049B538C925B3EF702FFF6FBD48DFE,0.666667,2.666667,3.333333,1.0,-0.106567,0.237967,0.333333,3


In [452]:
# Use numpy.where to adjust sentiment_score_summary based on star_sentiment
train_final['sentiment_score_summary'] = np.where(train_final['star_sentiment'] <0, -1,
                                                   np.where(train_final['star_sentiment'] >0, 1,
                                                            train_final['sentiment_score_summary']))

# Drop the star_sentiment column
train_final = train_final.drop('star_sentiment', axis=1)
train_final = train_final.sort_values(by=['sentiment_score_summary', 'awesomeness'], ascending=False)

In [453]:
train_final['asin_id'], map_table = pd.factorize(train_final['asin'])

In [454]:
map_table = train_final[['asin', 'asin_id']]

# Running the models 

In [457]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, KFold

# Train a Logistic Regression classifier on the training data
clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=num_cores)#precision = cross_val_score(clf, train_final_merged.drop(columns=['awesomeness']), train_final_merged['awesomeness'], cv=10, scoring='precision')
#recall = cross_val_score(clf, train_final_merged.drop(columns=['awesomeness']), train_final_merged['awesomeness'], cv=10, scoring='recall')

f1 = cross_val_score(clf, train_final.drop(columns=['asin', 'awesomeness']), train_final['awesomeness'], cv=KFold(n_splits=10, random_state=1, shuffle=True), scoring='f1')
precision = cross_val_score(clf, train_final.drop(columns=['asin', 'awesomeness']), train_final['awesomeness'], cv=KFold(n_splits=10, random_state=1, shuffle=True), scoring='precision')
recall = cross_val_score(clf, train_final.drop(columns=['asin', 'awesomeness']), train_final['awesomeness'], cv=KFold(n_splits=10, random_state=1, shuffle=True), scoring='recall')

# Print the scores
print("Precision: %0.2f (+/- %0.2f)" % (precision.mean(), precision.std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (recall.mean(), recall.std() * 2))
print("F1 Score: %0.2f (+/- %0.2f)" % (f1.mean(), f1.std() * 2))

Precision: 0.93 (+/- 0.01)
Recall: 0.91 (+/- 0.01)
F1 Score: 0.92 (+/- 0.01)


In [458]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score, f1_score

# Train a Gaussian Naive Bayes classifier on the training data
clf = GaussianNB()

f1 = cross_val_score(clf, train_final.drop(columns=['asin', 'awesomeness']), train_final['awesomeness'], cv=KFold(n_splits=10, random_state=1, shuffle=True), scoring='f1')
precision = cross_val_score(clf, train_final.drop(columns=['asin', 'awesomeness']), train_final['awesomeness'], cv=KFold(n_splits=10, random_state=1, shuffle=True), scoring='precision')
recall = cross_val_score(clf, train_final.drop(columns=['asin', 'awesomeness']), train_final['awesomeness'], cv=KFold(n_splits=10, random_state=1, shuffle=True), scoring='recall')

# Print the scores
print("Precision: %0.2f (+/- %0.2f)" % (precision.mean(), precision.std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (recall.mean(), recall.std() * 2))
print("F1 Score: %0.2f (+/- %0.2f)" % (f1.mean(), f1.std() * 2))

Precision: 0.81 (+/- 0.02)
Recall: 0.79 (+/- 0.01)
F1 Score: 0.80 (+/- 0.01)


In [460]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score, f1_score

# Train a Gaussian Naive Bayes classifier on the training data
clf = DecisionTreeClassifier()

f1 = cross_val_score(clf, train_final.drop(columns=['asin', 'awesomeness']), train_final['awesomeness'], cv=KFold(n_splits=10, random_state=1, shuffle=True), scoring='f1')
precision = cross_val_score(clf, train_final.drop(columns=['asin', 'awesomeness']), train_final['awesomeness'], cv=KFold(n_splits=10, random_state=1, shuffle=True), scoring='precision')
recall = cross_val_score(clf, train_final.drop(columns=['asin', 'awesomeness']), train_final['awesomeness'], cv=KFold(n_splits=10, random_state=1, shuffle=True), scoring='recall')

# Print the scores
print("Precision: %0.2f (+/- %0.2f)" % (precision.mean(), precision.std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (recall.mean(), recall.std() * 2))
print("F1 Score: %0.2f (+/- %0.2f)" % (f1.mean(), f1.std() * 2))

Precision: 0.91 (+/- 0.01)
Recall: 0.91 (+/- 0.01)
F1 Score: 0.91 (+/- 0.01)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
# Train a Logistic Regression classifier on the training data
clf = LogisticRegression(random_state=42, max_iter =400)
f1 = cross_val_score(clf, train_final.drop(columns=['asin', 'awesomeness']), train_final['awesomeness'], cv=KFold(n_splits=10, random_state=1, shuffle=True), scoring='f1')
precision = cross_val_score(clf, train_final.drop(columns=['asin', 'awesomeness']), train_final['awesomeness'], cv=KFold(n_splits=10, random_state=1, shuffle=True), scoring='precision')
recall = cross_val_score(clf, train_final.drop(columns=['asin', 'awesomeness']), train_final['awesomeness'], cv=KFold(n_splits=10, random_state=1, shuffle=True), scoring='recall')

# Print the scores
print("Precision: %0.2f (+/- %0.2f)" % (precision.mean(), precision.std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (recall.mean(), recall.std() * 2))
print("F1 Score: %0.2f (+/- %0.2f)" % (f1.mean(), f1.std() * 2))

In [ ]:
#Trying SVM on a resampled data

from sklearn import svm

sampled_data = train_final.sample(n=10000, random_state=42)

# Saw the kernel = 'linear' gave us the best fits
clf = svm.SVC(kernel='linear')

recall = cross_val_score(clf, train_final.drop(columns=['asin','awesomeness']), train_final['awesomeness'], cv=KFold(n_splits=10, random_state=1, shuffle=True), scoring='recall')
f1 = cross_val_score(clf, train_final.drop(columns=['asin', 'awesomeness']), train_final['awesomeness'], cv=KFold(n_splits=10, random_state=1, shuffle=True), scoring='f1')
precision = cross_val_score(clf, train_final.drop(columns=['asin','awesomeness']), train_final['awesomeness'], cv=KFold(n_splits=10, random_state=1, shuffle=True), scoring='precision')

print("Precision: %0.2f (+/- %0.2f)" % (precision.mean(), precision.std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (recall.mean(), recall.std() * 2))
print("F1 Score: %0.2f (+/- %0.2f)" % (f1.mean(), f1.std() * 2))

In [ ]:
#Trying SVM on a resampled data
from sklearn import svm

sampled_data = train_final.sample(n=10000, random_state=42)

# Saw the kernel = 'linear' gave us the best fits
clf = svm.SVC(kernel='linear')

#recall = cross_val_score(clf, train_final.drop(columns=['asin','awesomeness']), train_final['awesomeness'], cv=KFold(n_splits=10, random_state=1, shuffle=True), scoring='recall')
f1 = cross_val_score(clf, sampled_data.drop(columns=['asin', 'awesomeness']), sampled_data['awesomeness'], cv=KFold(n_splits=10, random_state=1, shuffle=True), scoring='f1')
#precision = cross_val_score(clf, train_final.drop(columns=['asin','awesomeness']), train_final['awesomeness'], cv=KFold(n_splits=10, random_state=1, shuffle=True), scoring='precision')

#print("Precision: %0.2f (+/- %0.2f)" % (precision.mean(), precision.std() * 2))
#print("Recall: %0.2f (+/- %0.2f)" % (recall.mean(), recall.std() * 2))
print("F1 Score: %0.2f (+/- %0.2f)" % (f1.mean(), f1.std() * 2))

In [ ]:
rom sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score

# Resample your data
sampled_data = train_final.sample(n=5000, random_state=42)

# Define your base estimator
base_estimator = DecisionTreeClassifiers()

# Define the AdaBoost classifier with the base estimator and desired number of estimators
adaboost = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=50, learning_rate = 1, random_state=42, algorithm='SAMME')

# Fit the AdaBoost classifier to your resampled data
adaboost.fit(sampled_data.drop(columns=['asin','awesomeness']), sampled_data['awesomeness'])

# Evaluate the performance of the AdaBoost classifier using cross-validation
f1 = cross_val_score(adaboost, sampled_data.drop(columns=['asin', 'awesomeness']), sampled_data['awesomeness'], cv=KFold(n_splits=10, random_state=1, shuffle=True), scoring='f1')
#precision = cross_val_score(adaboost, sampled_data.drop(columns=['asin', 'awesomeness']), sampled_data['awesomeness'], cv=KFold(n_splits=10, random_state=1, shuffle=True), scoring='precision')
#recall = cross_val_score(adaboost, sampled_data.drop(columns=['asin', 'awesomeness']), sampled_data['awesomeness'], cv=KFold(n_splits=10, random_state=1, shuffle=True), scoring='recall')

# Print the scores
#print("Precision: %0.2f (+/- %0.2f)" % (precision.mean(), precision.std() * 2))
#print("Recall: %0.2f (+/- %0.2f)" % (recall.mean(), recall.std() * 2))
print("F1 Score: %0.2f (+/- %0.2f)" % (f1.mean(), f1.std() * 2))

In [124]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score

# Resample your data
sampled_data = train_final.sample(n=10000, random_state=42)

# Define the XGBoost classifier with the desired parameters
xgb_model = xgb.XGBClassifier(n_estimators=50, learning_rate=1, random_state=42)

# Fit the XGBoost classifier to your resampled data
xgb_model.fit(sampled_data.drop(columns=['asin', 'awesomeness']), sampled_data['awesomeness'])

# Evaluate the performance of the XGBoost classifier using cross-validation
f1 = cross_val_score(xgb_model, sampled_data.drop(columns=['asin', 'awesomeness']), sampled_data['awesomeness'], cv=KFold(n_splits=10, random_state=1, shuffle=True), scoring='f1')
print("F1 Score: %0.2f (+/- %0.2f)" % (f1.mean(), f1.std() * 2))


F1 Score: 0.69 (+/- 0.05)


# Hyperparameter tuning 

In [56]:
from sklearn.model_selection import train_test_split

sampled_data = train_final.sample(n=2000, random_state=42)


# Split into 80-20 train/test split
train_data, test_data, train_labels, test_labels = train_test_split(train_final.drop(columns=['asin', 'awesomeness']), 
                                                                    train_final['awesomeness'], 
                                                                    test_size=0.3, 
                                                                random_state=42, stratify = train_final['awesomeness'])
#for the smaller dataset
train_data_s, test_data_s, train_labels_s, test_labels_s = train_test_split(sampled_data.drop(columns=['asin', 'awesomeness']), 
                                                                    sampled_data['awesomeness'], 
                                                                    test_size=0.3, 
                                                                random_state=42, stratify = sampled_data['awesomeness'])


In [59]:
#Hyperparameter tuning the SVM 
from sklearn.model_selection import GridSearchCV
  
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 20], 
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ['linear']} 
  
grid = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 3, scoring='f1')
  
# fitting the model for grid search
grid.fit(train_data_s, train_labels_s)

# print best parameter after tuning
print(grid.best_params_)
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.697 total time=   0.9s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.708 total time=   0.8s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.728 total time=   0.8s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.731 total time=   0.8s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.738 total time=   0.8s
[CV 1/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.697 total time=   1.0s
[CV 2/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.708 total time=   1.1s
[CV 3/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.728 total time=   0.9s
[CV 4/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.731 total time=   0.9s
[CV 5/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.738 total time=   0.8s
[CV 1/5] END ..C=0.1, gamma=0.01, kernel=linear;, score=0.697 total time=   0.8s
[CV 2/5] END ..C=0.1, gamma=0.01, kernel=linear;

KeyboardInterrupt: 

In [12]:
from sklearn.model_selection import GridSearchCV, train_test_split

#RF 
param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 8],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

# Train a random forest classifier using grid search to find the best hyperparameters
clf = RandomForestClassifier()
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(train_data, train_labels)

# print best parameter after tuning
print(grid_search.best_params_)
# print how our model looks after hyper-parameter tuning
print(grid_search.best_estimator_)

Best parameters: {'max_depth': 20, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 8, 'n_estimators': 200}
Best cross-validation score: 0.7181


NameError: name 'accuracy_score' is not defined

In [ ]:
#Decision Tree 
param_grid = {
    'max_depth': [2, 3, 5, 10, 20],
    'min_samples_leaf': [5, 10, 20, 50, 100],
    'criterion': ["gini", "entropy"]
}

clf = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5)
grid_search.fit(train_data, train_labels)

# print best parameter after tuning
print(grid_search.best_params_)
# print how our model looks after hyper-parameter tuning
print(grid_search.best_estimator_)

# Fitting the Model

In [441]:
clf = LogisticRegression(random_state=42, max_iter = 400)
clf.fit(train_final.drop(columns=['asin', 'awesomeness']), train_final['awesomeness'])

RandomForestClassifier(n_jobs=8, random_state=42)

# Testing 

In [201]:
id_counts = test_review['reviewerID'].value_counts()
user_id_dict = id_counts.to_dict()
test_review['reviewerNumber'] = test_review['reviewerID'].map(user_id_dict)

In [202]:
test = test_review.merge(test_product, left_on = 'asin', right_on = 'asin')
len(test)
test['reviewText'] = test['reviewText'].astype(str)
test['reviewText']=test['reviewText'].apply(remove_stopwords)
test['summary'] = test['summary'].astype(str)
test['summary']=test['summary'].apply(remove_stopwords)

In [203]:
#need to pip install vaderSentiment in terminal
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def get_sentiment_score(text):
    sentiment_scores = analyzer.polarity_scores(text)
    return sentiment_scores['compound']

# apply the function to the 'reviewText' column: takes ~5-10 min to run 
test['sentiment_score_review'] = test['reviewText'].apply(get_sentiment_score)
test['sentiment_score_summary'] = test['summary'].apply(get_sentiment_score)

In [204]:
#changed verified to 1 and 0, change vote none to 0, change image to binary
test["verified"] = test["verified"].astype(int)
test["vote"] = test["vote"].replace(np.nan,0) 
test['image'] = test['image'].fillna(0)
test['image'] = test['image'].apply(str)
test['image'] = test['image'].replace(to_replace=r'[^0-9]', value=1, regex=True)
test = test[['asin', 'verified', 'vote', 'image', 'reviewerNumber', 'sentiment_score_review', 'sentiment_score_summary']] 
#Change to float
test['vote'] = test['vote'].apply(str)
test['vote'] = test['vote'].apply(lambda x: float(x.replace(',', '')))
test['image'] = test['image'].astype(float)

In [444]:
test = test.groupby('asin').mean().reset_index()
test_counts = test.groupby('asin').size().reset_index(name='num_reviews_per_product')
test_final = test.groupby('asin').mean().reset_index().merge(test_counts, on='asin')
test_final

,asin,verified,vote,image,reviewerNumber,sentiment_score_review,sentiment_score_summary,num_reviews_per_product
0,000254CDA01E5AE839E7EE3F6FC1D2F1,1.0,0.000000,0.0,2.714286,0.665057,0.377957,1
1,00027761289A98A2884B5F152D3DBF91,1.0,0.000000,0.0,3.000000,0.273200,0.273200,1
2,0002E81AED1B9840434EE28E39876BE4,1.0,2.000000,0.0,2.250000,-0.502100,0.044175,1
3,00049B538C925B3EF702FFF6FBD48DFE,1.0,0.000000,0.0,1.000000,0.624900,0.000000,1
4,000564E0025921626AF9AD7CE58C4DE9,1.0,0.000000,0.0,2.000000,0.593050,0.318450,1
...,...,...,...,...,...,...,...,...
65590,FFFA1AABE8829718F285725C4D7222A8,1.0,0.000000,0.0,4.000000,0.906200,0.599400,1
65591,FFFAB16CD5964F60F014EFAA1210AFEE,1.0,0.000000,0.0,3.000000,0.919550,0.448950,1
65592,FFFAB94C16EAEC9E1DEBF24E231C53BB,1.0,0.000000,0.0,1.800000,0.520440,0.140880,1
65593,FFFB0C64CBC232115CE8FFF2BE870122,1.0,8.666667,0.0,2.333333,0.796700,0.575300,1


In [445]:
test_final = pd.merge(test_final, map_table, on='asin')
test_final = test_final.fillna(0)
test_final

,asin,verified,vote,image,reviewerNumber,sentiment_score_review,sentiment_score_summary,num_reviews_per_product,asin_id
0,000254CDA01E5AE839E7EE3F6FC1D2F1,1.0,0.000000,0.0,2.714286,0.665057,0.377957,1,66192
1,00027761289A98A2884B5F152D3DBF91,1.0,0.000000,0.0,3.000000,0.273200,0.273200,1,71503
2,0002E81AED1B9840434EE28E39876BE4,1.0,2.000000,0.0,2.250000,-0.502100,0.044175,1,77737
3,00049B538C925B3EF702FFF6FBD48DFE,1.0,0.000000,0.0,1.000000,0.624900,0.000000,1,77721
4,000564E0025921626AF9AD7CE58C4DE9,1.0,0.000000,0.0,2.000000,0.593050,0.318450,1,47678
...,...,...,...,...,...,...,...,...,...
65078,FFFA1AABE8829718F285725C4D7222A8,1.0,0.000000,0.0,4.000000,0.906200,0.599400,1,40497
65079,FFFAB16CD5964F60F014EFAA1210AFEE,1.0,0.000000,0.0,3.000000,0.919550,0.448950,1,15349
65080,FFFAB94C16EAEC9E1DEBF24E231C53BB,1.0,0.000000,0.0,1.800000,0.520440,0.140880,1,19530
65081,FFFB0C64CBC232115CE8FFF2BE870122,1.0,8.666667,0.0,2.333333,0.796700,0.575300,1,23262


In [446]:
test_outcome=test_final[['asin', 'vote']]
logreg_probs = clf.predict_proba(test_final.drop(columns=['asin', 'image']))[:,1]
test_outcome['vote'] = np.round(logreg_probs)

In [447]:
test_outcome.head()

,asin,vote
0,000254CDA01E5AE839E7EE3F6FC1D2F1,0.0
1,00027761289A98A2884B5F152D3DBF91,1.0
2,0002E81AED1B9840434EE28E39876BE4,0.0
3,00049B538C925B3EF702FFF6FBD48DFE,0.0
4,000564E0025921626AF9AD7CE58C4DE9,1.0


In [448]:
pred_groundtruth = pd.read_json(filepath + '/Toys_and_Games_test1_labels.json')
pred_total = pd.merge(pred_groundtruth, test_outcome, on='asin')
pred_total.head()

,asin,awesomeness,vote
0,000254CDA01E5AE839E7EE3F6FC1D2F1,1,0.0
1,00027761289A98A2884B5F152D3DBF91,0,1.0
2,000BFD712DBCE3DDD81CA2B0E306F619,1,0.0
3,0067EBC19C5C93CFDB9F3AF801519FB1,1,0.0
4,2711AD144669B4EE24FDB75D43F17F17,1,0.0


In [449]:
from sklearn.metrics import accuracy_score, f1_score
print(accuracy_score(pred_total.awesomeness, pred_total.vote))
print(f1_score(pred_total.awesomeness, pred_total.vote))

0.5277261343207904
0.5146457389189787


In [100]:
import json
json_str = test_outcome.to_json()

json_obj = json.loads(json_str)

# Open a file in write mode and write the JSON object to it
with open('prediction.json', 'w') as f:
    json.dump(json_obj, f)